In [ ]:
from PIL import Image
import numpy as np
import os
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms,utils
import pickle
from skimage.filters.rank import entropy
from skimage.morphology import disk
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import LeaveOneOut
from IPython import display
from sklearn.neural_network import MLPClassifier
from sklearn import metrics

import tensorflow as tf

In [ ]:
# Load data using torch

mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())
batch_size = 100
test_loader = torch.utils.data.DataLoader(dataset=mnist_testset, batch_size=batch_size, shuffle=False)

# Load the ResNet-18 model and delete the last FC layer to get a feature extractor
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_conv = nn.Sequential(*list(torchvision.models.resnet18(pretrained=True).children())[:-1]).double()
for param in model_conv.parameters():
        param.requires_grad = False
model_conv = model_conv.to(device)
model_conv.eval()

# Extract the features
for batch_idx, (x, target) in enumerate(test_loader):
    x = x.repeat(1, 3, 1, 1).double() # Fit the greyscale data to the momdel
    print("ResNet "+str(batch_idx)+"/"+str(100))
    if batch_idx == 0:
        test_feat = np.squeeze(model_conv(x).numpy())
        test_label = target
    else:
        test_feat = np.concatenate( ( test_feat, np.squeeze(model_conv(x).numpy()) ), axis=0 )
        test_label = np.concatenate((test_label, target), axis=0)


In [ ]:
#Visualization using PCA

from sklearn import decomposition
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

pca = decomposition.PCA(n_components=3)
feat_PCA = pca.fit_transform(test_feat)

# Choose the first 500 points and plot the points of digit 0, 7, 8
for i in range(0,500):
    if test_label[i] == 0:
        ax.scatter(feat_PCA[i,0], feat_PCA[i,1], feat_PCA[i,2], c='r', marker='o')
    if test_label[i] == 7:
        ax.scatter(feat_PCA[i,0], feat_PCA[i,1], feat_PCA[i,2], c='b', marker='o')
    if test_label[i] == 8:
        ax.scatter(feat_PCA[i,0], feat_PCA[i,1], feat_PCA[i,2], c='y', marker='o')

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()

In [ ]:
# feature extraction of train data

mnist_testset = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())

batch_size = 100
train_loader = torch.utils.data.DataLoader(dataset=mnist_testset, batch_size=batch_size, shuffle=False)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_conv = nn.Sequential(*list(torchvision.models.resnet18(pretrained=True).children())[:-1]).double()
for param in model_conv.parameters():
        param.requires_grad = False
model_conv = model_conv.to(device)
model_conv.eval()

for batch_idx, (x, target) in enumerate(train_loader):
    x = x.repeat(1, 3, 1, 1).double()
    print("ResNet "+str(batch_idx)+"/"+str(len(train_loader)))
    if batch_idx == 0:
        train_feat = np.squeeze(model_conv(x).numpy())
        train_label = target
    else:
        train_feat = np.concatenate((train_feat, np.squeeze(model_conv(x).numpy())), axis=0)
        train_label = np.concatenate((train_label, target), axis=0)
        

In [ ]:
# Train SVM based on the features of the train data

from sklearn.svm import SVC
clf = SVC(gamma='auto')
clf.fit(train_feat, train_label) 

In [ ]:
# Classification and print accuracy

test_result = clf.predict(test_feat)

correct_count = 0
for i in np.arange(0,len(test_label)):
    if test_result[i] == test_label[i]:
        correct_count = correct_count + 1
print('accuracy: '+str(correct_count/len(test_label)))
        